In [4]:
import os
import json
import random
from typing import Dict, List, Any

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    TrainingArguments,
    Trainer,
)
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


In [5]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
cache_dir = "/home/ubuntu/marbles/Projects/narrative-dataset/model_cache-2"
model_name = "unsloth/Llama-3.3-70B-Instruct"
train_dataset_path = "./tp_data_train.json"
val_dataset_path = "./tp_data_val.json"
test_dataset_path = "./tp_data_test.json"
lora_save_name = "70b-4bit-lora-r128"
lora_dim = 128

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    cache_dir=cache_dir,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_dim, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 8/8 [00:43<00:00,  5.44s/it]
Unsloth 2025.1.5 patched 80 layers with 80 QKV layers, 80 O layers and 80 MLP layers.


In [7]:
def formatting_prompts_func(examples: List[Dict[str, Any]]) -> List[str]:
    """
    Format prompts by appending EOS token to complete-text.
    """
    output_texts = []
    for example in examples:
        text = example['complete-text'] + tokenizer.eos_token
        output_texts.append(text)
    return output_texts

def load_json_data(json_path: str) -> List[Dict[str, Any]]:
    """
    Load narrative data from a JSON file.
    This function wraps multiple JSON objects into a list if they aren't already.
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        content = f.read().strip()
        try:
            data = json.loads(content)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            data = []
    
    return data

train_data = load_json_data(train_dataset_path)
formatted_train_texts = formatting_prompts_func(train_data)
train_dataset = Dataset.from_list([{"text": text} for text in formatted_train_texts])

# Load and format validation data  
val_data = load_json_data(val_dataset_path)
formatted_val_texts = formatting_prompts_func(val_data)
val_dataset = Dataset.from_list([{"text": text} for text in formatted_val_texts])


In [8]:
# Find the longest text in the dataset
max_len = 0
longest_text = ""
for example in val_dataset:
    if len(example["text"]) > max_len:
        max_len = len(example["text"])
        longest_text = example["text"]
len(longest_text)

15701

In [9]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        evaluation_strategy = "steps",
        eval_steps = 5,
    ),
)

/home/ubuntu/marbles/Projects/miniconda/envs/unsloth_env/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/ubuntu/marbles/Projects/miniconda/envs/unsloth_env/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Map (num_proc=2): 100%|██████████| 88/88 [00:01<00:00, 47.69 examples/s]


In [10]:
from unsloth import unsloth_train
trainer_stats = unsloth_train(trainer)
model.save_pretrained(lora_save_name)  # Local saving
tokenizer.save_pretrained(lora_save_name)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 351 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 1,656,750,080


Step,Training Loss,Validation Loss
5,1.458400,1.453833
10,1.205700,1.256121
15,1.101700,1.106556
20,1.057400,1.061546
25,1.033400,1.049191
30,1.041800,1.045420


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


('70b-4bit-lora-r128/tokenizer_config.json',
 '70b-4bit-lora-r128/special_tokens_map.json',
 '70b-4bit-lora-r128/tokenizer.json')

In [11]:
val_dataset[0]["text"].split("### Response:\n")[1]

IndexError: list index out of range

In [1]:

from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "70b-4bit-lora-r128", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    cache_dir=cache_dir,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/marbles/Projects/miniconda/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# if True:
#     from unsloth import FastLanguageModel
#     from transformers import AutoTokenizer
    
#     # Load the base model first
#     model = None
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "unsloth/Llama-3.3-70B-Instruct",
#         max_seq_length = max_seq_length,
#         dtype = dtype,
#         load_in_4bit = load_in_4bit,
#         # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
#         cache_dir=cache_dir,
#     )
    
#     # Load the LoRA adapter weights
#     model = FastLanguageModel.get_peft_model(model)
#     model.load_adapter("lora_model")
    
#     # Load tokenizer separately
#     tokenizer = AutoTokenizer.from_pretrained("lora_model")
    
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    val_data[0]["query"]
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 8192)

<|begin_of_text|>### INSTRUCTIONS
        You are a helpful assistant that identifies and explains turning points in a narrative. You are given:
        - The story, broken down into numbered sentences.
        - The definitions of each of the five turning points (Opportunity, Change of Plans, Point of No Return, Major Setback, Climax).
        - Ground truth turning point indices for this story.
        
        ### TURNING POINT DEFINITIONS
        - **Opportunity** – Introductory event that occurs after presenting the setting and background of the main characters.
        - **Change of Plans** – Event where the main goal of the story is defined, starting the main action.
        - **Point of No Return** – Event that pushes the main character(s) to fully commit to their goal.
        - **Major Setback** – Event where things fall apart temporarily or permanently.
        - **Climax** – Final event/resolution of the main story (the "biggest spoiler").

### INPUT 
        Story, broken 

 ### Task 1: Identify the Protagonist or Central Group of Characters
The central group of characters in the story consists of four friends: Sharon, Carol, Vivian, and Diane. They are the main focus of the narrative, and the story revolves around their experiences, relationships, and personal growth.

### Task 2: Summarize Each Major Event and Explore How Those Events Change the Protagonist's Condition
1. **Introduction and Background**: The story begins with the four friends meeting in person after a period of virtual meetings due to COVID-19. This setting establishes their friendship and individual backgrounds, including significant life changes such as Sharon's retirement, Carol's husband's heart attack, and Vivian's engagement.
2. **The Trip Proposal**: Carol suggests a bachelorette trip to Italy, which initially meets resistance but eventually gains traction as a way for them to reconnect and celebrate Vivian's upcoming wedding. This proposal sets the stage for their adventures and

In [2]:
import re

import json

from typing import Dict, List, Any, Optional
def extract_tp_from_text(text: str) -> Optional[Dict[str, Any]]:
    """
    Extract the turning points from the model's generated text if it follows
    the $\boxed{...}$ format containing a JSON structure.
    """
    pattern = r'\$\\boxed\{(.*?)\}\$'
    match = re.search(pattern, text)
    if match:
        json_str = match.group(1)
        # Convert the string to a standard JSON format
        json_str = '{' + json_str.replace('"', '"') + '}'
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None
    return None


### Final Turning Point Locations

extract_tp_from_text("""Based on the analysis, the final turning point locations in JSON format are:

```json
{
  ""tp1"": 7,
  ""tp2"": 10,
  ""tp3"": 14,
  ""tp4"": 19,
  ""tp5"": 35
}
```""")